In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
aapl_text_direction_date.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_processedText_direction_date.csv
amzn_title_text_label.csv
lstm_amzn_fulltext/
text_kfold_aapl.csv


In [ ]:
!pip install keras
!pip install matplotlib-venn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import TimeSeriesSplit, train_test_split, KFold, cross_validate

import requests
import io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def split_generator(df):
  cv=[]
  df['prediction_date'] = pd.to_datetime(df['prediction_date'], errors='coerce')
  groups = df.groupby(df['prediction_date'].dt.to_period("M")).groups
  split_groups = [value for (key, value) in groups.items()] 

  print('spilt number:' + str(len(split_groups)))
  for i in range(len(split_groups)):
    data_index=[]
    for j in range(len(split_groups)):
      if i != j:
        data_index.extend(split_groups[j].tolist())
    cv.append((data_index, split_groups[i].tolist()))
    # yield data_index, split_groups[i].tolist()
  
  return cv

In [ ]:
# df_aapl = pd.read_csv('Aapl_SentimentAggMeanPriceHistDirection.csv', nrows=5000)
# df_amzn = pd.read_csv('Amzn_SentimentAggMeanPriceHistDirection.csv', nrows=5000)
# df = [df_aapl, df_amzn]
# df = pd.concat(df, ignore_index=True)
# df = df.replace(-1, 0)
# df
df_aapl = pd.read_csv('Aapl_NewsPriceHistDirection.csv', nrows=10000)
df_amzn = pd.read_csv('Amzn_NewsPriceHistDirection.csv', nrows=10000)

In [ ]:
df_amzn

,uuid,author,url,title,language,text,published,title_aapl,text_aapl,title_amzn,...,prevClose_16,prevClose_17,prevClose_18,prevClose_19,prevClose_20,prevClose_21,prevClose_22,prevClose_23,prevClose_24,prevClose_25
0,3b7327f6858be72e780b02b863d941fe7ce81eab,NaN,https://money.cnn.com/2018/09/05/investing/pre...,6 things to know before the opening bell,english,Click chart for more in-depth data. 1. Big Tec...,2018-09-05 05:12:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
1,8a3015a274df99d231fcf7ff3eb38e2660a984ff,Michael Wiggins De Oliveira,https://seekingalpha.com/article/4203857-netfl...,"Netflix And The Competition - Netflix, Inc. (N...",english,Investment Thesis Netflix ( NFLX ) is a remark...,2018-09-04 15:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
2,df5883d078ebfbe015e1e717002e4b889647dbee,Larry Ramer,https://investorplace.com/2018/09/pandora-stoc...,Pandora Stock Gets a Boost — Buy Now,english,MannKind Surges More Than 50% >>> READ MORE\nB...,2018-09-04 16:22:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
3,861c19bff56d09f97b6798f5fbe374061e8aa55b,morningstar.com,http://news.morningstar.com/all/dow-jones/us-m...,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,english,"By Mark DeCambre, MarketWatch , Ryan Vlastelic...",2018-09-05 00:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
4,cb960bebc2bf78d327026127c9fe919a4159e80d,morningstar.com,http://news.morningstar.com/all/benzinga/BenzB...,"New Month, Same Old Story: Trade Fears Persist...",english,"It may be a new month, but the same trade conc...",2018-09-05 00:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,abd5d4421293377003c20449f96240c6f4b5e3fd,Barron's,https://www.nasdaq.com/article/tech-today-chip...,"Tech Today: Chips' Tariff Risk, Spotify a Must...",english,Here are some things going on today in the wor...,2018-07-23 16:22:00+00:00,False,True,False,...,1698.76,1700.30,1660.99,1690.00,1661.98,1715.06,1728.25,1749.56,1734.00,1723.17
9996,50d332dc735980d49b3c5fefec97c6bece3e9d3b,Paresh Dave and Arjun Panchadar,https://sg.finance.yahoo.com/news/alphabet-tro...,Alphabet trounces profit estimates as costs co...,english,1 / 2 The logo of Google is pictured during th...,2018-07-24 03:50:00+00:00,False,True,False,...,1698.76,1700.30,1660.99,1690.00,1661.98,1715.06,1728.25,1749.56,1734.00,1723.17
9997,d186ac29741002a2b3813593c83ad94f9b65472c,Jonathan Vanian,http://fortune.com/2018/07/23/amazon-echo-alex...,Amazon Debuts Alexa-Powered Equalizer for Echo...,english,"By Jonathan Vanian July 23, 2018\nYou can fina...",2018-07-24 01:18:00+00:00,False,True,True,...,1698.76,1700.30,1660.99,1690.00,1661.98,1715.06,1728.25,1749.56,1734.00,1723.17
9998,11319b499842b4dc0a8ce8cac3313116c0a513f8,Morningstar Equity Analysts,http://www.morningstar.co.uk/uk/news/169073/an...,"Analysts Upgrade Netflix, but it Remains Overv...",english,Equity Research & Insights Analysts Upgrade N...,2018-07-24 10:02:00+00:00,False,True,False,...,1698.76,1700.30,1660.99,1690.00,1661.98,1715.06,1728.25,1749.56,1734.00,1723.17


In [ ]:
df_amzn.columns

Index(['uuid', 'author', 'url', 'title', 'language', 'text', 'published',
       'title_aapl', 'text_aapl', 'title_amzn', 'text_amzn', 'title_posSent',
       'title_neg_sent', 'title_net_sent', 'text_pos_sent', 'text_neg_sent',
       'text_net_sent', 'entity_pos_sent', 'entity_neg_sent',
       'entity_net_sent', 'title_entities', 'text_entities', 'prediction_date',
       'open', 'high', 'low', 'close', 'volume', 'direction', 'pct_chng',
       'prevClose_1', 'prevClose_2', 'prevClose_3', 'prevClose_4',
       'prevClose_5', 'prevClose_6', 'prevClose_7', 'prevClose_8',
       'prevClose_9', 'prevClose_10', 'prevClose_11', 'prevClose_12',
       'prevClose_13', 'prevClose_14', 'prevClose_15', 'prevClose_16',
       'prevClose_17', 'prevClose_18', 'prevClose_19', 'prevClose_20',
       'prevClose_21', 'prevClose_22', 'prevClose_23', 'prevClose_24',
       'prevClose_25'],
      dtype='object')

In [ ]:
amzn_dataset = df_amzn[['prevClose_1', 'prevClose_2', 'prevClose_3', 'prevClose_4',
       'prevClose_5', 'prevClose_6', 'prevClose_7', 'prevClose_8',
       'prevClose_9', 'prevClose_10', 'prevClose_11', 'prevClose_12',
       'prevClose_13', 'prevClose_14', 'prevClose_15', 'prevClose_16',
       'prevClose_17', 'prevClose_18', 'prevClose_19', 'prevClose_20',
       'prevClose_21', 'prevClose_22', 'prevClose_23', 'prevClose_24',
       'prevClose_25', 'prediction_date', 'direction']].sort_values(by=['prediction_date'])

In [ ]:
amzn_dataset

,prevClose_1,prevClose_2,prevClose_3,prevClose_4,prevClose_5,prevClose_6,prevClose_7,prevClose_8,prevClose_9,prevClose_10,...,prevClose_18,prevClose_19,prevClose_20,prevClose_21,prevClose_22,prevClose_23,prevClose_24,prevClose_25,prediction_date,direction
7838,1173.50,1163.58,1164.00,1168.00,1161.32,1160.00,1152.99,1140.97,1132.97,1161.50,...,1126.01,1129.20,1136.89,1126.30,1135.75,1127.99,1124.78,1125.60,2017-12-15,1
7662,1174.40,1186.43,1190.58,1178.15,1173.50,1163.58,1164.00,1168.00,1161.32,1160.00,...,1196.10,1186.41,1156.19,1138.58,1126.01,1129.20,1136.89,1126.30,2017-12-21,-1
7663,1174.40,1186.43,1190.58,1178.15,1173.50,1163.58,1164.00,1168.00,1161.32,1160.00,...,1196.10,1186.41,1156.19,1138.58,1126.01,1129.20,1136.89,1126.30,2017-12-21,-1
7661,1174.78,1174.40,1186.43,1190.58,1178.15,1173.50,1163.58,1164.00,1168.00,1161.32,...,1192.33,1196.10,1186.41,1156.19,1138.58,1126.01,1129.20,1136.89,2017-12-22,-1
7656,1168.21,1174.78,1174.40,1186.43,1190.58,1178.15,1173.50,1163.58,1164.00,1168.00,...,1161.03,1192.33,1196.10,1186.41,1156.19,1138.58,1126.01,1129.20,2017-12-26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360,1709.98,1684.04,1612.87,1633.73,1670.81,1649.99,1642.10,1633.00,1694.82,1687.02,...,1629.03,1575.66,1500.53,1505.00,1503.37,1477.95,1457.86,1469.10,2019-02-01,-1
3361,1709.98,1684.04,1612.87,1633.73,1670.81,1649.99,1642.10,1633.00,1694.82,1687.02,...,1629.03,1575.66,1500.53,1505.00,1503.37,1477.95,1457.86,1469.10,2019-02-01,-1
3363,1709.98,1684.04,1612.87,1633.73,1670.81,1649.99,1642.10,1633.00,1694.82,1687.02,...,1629.03,1575.66,1500.53,1505.00,1503.37,1477.95,1457.86,1469.10,2019-02-01,-1
3351,1709.98,1684.04,1612.87,1633.73,1670.81,1649.99,1642.10,1633.00,1694.82,1687.02,...,1629.03,1575.66,1500.53,1505.00,1503.37,1477.95,1457.86,1469.10,2019-02-01,-1


In [ ]:
amzn_dataset.drop(columns=['prediction_date'], inplace=True)

In [ ]:
Amzn_X, Amzn_Y = amzn_dataset.drop(columns=['direction']), (amzn_dataset['direction'] + 1) / 2

In [ ]:
Amzn_X

,prevClose_1,prevClose_2,prevClose_3,prevClose_4,prevClose_5,prevClose_6,prevClose_7,prevClose_8,prevClose_9,prevClose_10,...,prevClose_16,prevClose_17,prevClose_18,prevClose_19,prevClose_20,prevClose_21,prevClose_22,prevClose_23,prevClose_24,prevClose_25
7838,1173.50,1163.58,1164.00,1168.00,1161.32,1160.00,1152.99,1140.97,1132.97,1161.50,...,1156.19,1138.58,1126.01,1129.20,1136.89,1126.30,1135.75,1127.99,1124.78,1125.60
7662,1174.40,1186.43,1190.58,1178.15,1173.50,1163.58,1164.00,1168.00,1161.32,1160.00,...,1161.03,1192.33,1196.10,1186.41,1156.19,1138.58,1126.01,1129.20,1136.89,1126.30
7663,1174.40,1186.43,1190.58,1178.15,1173.50,1163.58,1164.00,1168.00,1161.32,1160.00,...,1161.03,1192.33,1196.10,1186.41,1156.19,1138.58,1126.01,1129.20,1136.89,1126.30
7661,1174.78,1174.40,1186.43,1190.58,1178.15,1173.50,1163.58,1164.00,1168.00,1161.32,...,1175.50,1161.03,1192.33,1196.10,1186.41,1156.19,1138.58,1126.01,1129.20,1136.89
7656,1168.21,1174.78,1174.40,1186.43,1190.58,1178.15,1173.50,1163.58,1164.00,1168.00,...,1161.50,1175.50,1161.03,1192.33,1196.10,1186.41,1156.19,1138.58,1126.01,1129.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360,1709.98,1684.04,1612.87,1633.73,1670.81,1649.99,1642.10,1633.00,1694.82,1687.02,...,1656.60,1653.00,1629.03,1575.66,1500.53,1505.00,1503.37,1477.95,1457.86,1469.10
3361,1709.98,1684.04,1612.87,1633.73,1670.81,1649.99,1642.10,1633.00,1694.82,1687.02,...,1656.60,1653.00,1629.03,1575.66,1500.53,1505.00,1503.37,1477.95,1457.86,1469.10
3363,1709.98,1684.04,1612.87,1633.73,1670.81,1649.99,1642.10,1633.00,1694.82,1687.02,...,1656.60,1653.00,1629.03,1575.66,1500.53,1505.00,1503.37,1477.95,1457.86,1469.10
3351,1709.98,1684.04,1612.87,1633.73,1670.81,1649.99,1642.10,1633.00,1694.82,1687.02,...,1656.60,1653.00,1629.03,1575.66,1500.53,1505.00,1503.37,1477.95,1457.86,1469.10


In [ ]:
Amzn_Y

7838    1.0
7662    0.0
7663    0.0
7661    0.0
7656    1.0
       ... 
3360    0.0
3361    0.0
3363    0.0
3351    0.0
3391    0.0
Name: direction, Length: 10000, dtype: float64

In [ ]:
!pip install scikeras
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

#def lstm_model(TrainX, TrainY):
def lstm_model(shape):
  layers = [LSTM(128,input_shape=(shape[1],1),dropout=0.2), Dense(128, activation='relu'), Dense(1)]
  model = tf.keras.models.Sequential(layers)
  #model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
  #keras_clf = KerasClassifier(model = model, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'], optimizer=tf.keras.optimizers.Adam(), epochs=20, batch_size=64, verbose=1,validation_split=0.2)

  #predictions = cross_val_predict(keras_clf, TrainX, trainY, cv=3)
  return model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(Amzn_X, Amzn_Y, test_size=0.2, random_state=56)

In [ ]:
TrainX = X_train.to_numpy().reshape((X_train.shape[0], X_train.shape[1], 1))

In [ ]:
TestX = X_test.to_numpy().reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
Y_train

4530    1.0
8869    0.0
6978    1.0
273     1.0
8287    0.0
       ... 
2063    1.0
2832    0.0
8518    1.0
4777    0.0
9227    1.0
Name: direction, Length: 8000, dtype: float64

In [ ]:
model = lstm_model(X_train.shape)

In [ ]:
model.fit(TrainX, Y_train, epochs=10, batch_size=64, verbose=1,validation_split=0.2)

Epoch 1/10
100/100 [==============================] - 7s 52ms/step - loss: 0.6897 - accuracy: 0.4659 - val_loss: 0.7054 - val_accuracy: 0.4906
Epoch 2/10
100/100 [==============================] - 5s 45ms/step - loss: 0.6882 - accuracy: 0.4659 - val_loss: 0.7038 - val_accuracy: 0.4906
Epoch 3/10
100/100 [==============================] - 5s 49ms/step - loss: 0.6854 - accuracy: 0.4658 - val_loss: 0.7017 - val_accuracy: 0.4906
Epoch 4/10
100/100 [==============================] - 5s 45ms/step - loss: 0.6869 - accuracy: 0.4781 - val_loss: 0.7029 - val_accuracy: 0.4906
Epoch 5/10
100/100 [==============================] - 5s 48ms/step - loss: 0.6852 - accuracy: 0.4811 - val_loss: 0.6998 - val_accuracy: 0.4906
Epoch 6/10
100/100 [==============================] - 9s 87ms/step - loss: 0.6828 - accuracy: 0.4952 - val_loss: 0.7143 - val_accuracy: 0.5138
Epoch 7/10
100/100 [==============================] - 8s 80ms/step - loss: 0.6831 - accuracy: 0.5003 - val_loss: 0.7065 - val_accuracy: 0.5400

In [ ]:
model.fit(TrainX, Y_train, epochs=20, batch_size=64, verbose=1,validation_split=0.2)

Epoch 1/20
100/100 [==============================] - 7s 71ms/step - loss: 0.6817 - accuracy: 0.5188 - val_loss: 0.7016 - val_accuracy: 0.5400
Epoch 2/20
100/100 [==============================] - 6s 62ms/step - loss: 0.6816 - accuracy: 0.5167 - val_loss: 0.7183 - val_accuracy: 0.5275
Epoch 3/20
100/100 [==============================] - 5s 46ms/step - loss: 0.6811 - accuracy: 0.5247 - val_loss: 0.6961 - val_accuracy: 0.5400
Epoch 4/20
100/100 [==============================] - 4s 45ms/step - loss: 0.6820 - accuracy: 0.5208 - val_loss: 0.6930 - val_accuracy: 0.5400
Epoch 5/20
100/100 [==============================] - 5s 45ms/step - loss: 0.6810 - accuracy: 0.5219 - val_loss: 0.7016 - val_accuracy: 0.5400
Epoch 6/20
100/100 [==============================] - 5s 45ms/step - loss: 0.6798 - accuracy: 0.5309 - val_loss: 0.6972 - val_accuracy: 0.5400
Epoch 7/20
100/100 [==============================] - 4s 45ms/step - loss: 0.6809 - accuracy: 0.5247 - val_loss: 0.7089 - val_accuracy: 0.5337

In [ ]:
eva = model.evaluate(TestX, Y_test, verbose=2)

63/63 - 1s - loss: 0.6939 - accuracy: 0.5545 - 1s/epoch - 20ms/step


In [ ]:
aapl_dataset = df_aapl[['prevClose_1', 'prevClose_2', 'prevClose_3', 'prevClose_4',
       'prevClose_5', 'prevClose_6', 'prevClose_7', 'prevClose_8',
       'prevClose_9', 'prevClose_10', 'prevClose_11', 'prevClose_12',
       'prevClose_13', 'prevClose_14', 'prevClose_15', 'prevClose_16',
       'prevClose_17', 'prevClose_18', 'prevClose_19', 'prevClose_20',
       'prevClose_21', 'prevClose_22', 'prevClose_23', 'prevClose_24',
       'prevClose_25', 'prediction_date', 'direction']].sort_values(by=['prediction_date'])

In [ ]:
aapl_dataset

,prevClose_1,prevClose_2,prevClose_3,prevClose_4,prevClose_5,prevClose_6,prevClose_7,prevClose_8,prevClose_9,prevClose_10,...,prevClose_18,prevClose_19,prevClose_20,prevClose_21,prevClose_22,prevClose_23,prevClose_24,prevClose_25,prediction_date,direction
5658,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08,2018-08-02,1
5766,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08,2018-08-02,1
5765,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08,2018-08-02,1
5764,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08,2018-08-02,1
5763,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08,2018-08-02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9613,165.66,160.54,155.92,157.67,152.44,153.88,153.15,156.80,155.65,153.99,...,148.28,142.50,146.09,157.93,156.18,156.05,156.30,146.59,2019-01-31,1
9614,165.66,160.54,155.92,157.67,152.44,153.88,153.15,156.80,155.65,153.99,...,148.28,142.50,146.09,157.93,156.18,156.05,156.30,146.59,2019-01-31,1
9615,165.66,160.54,155.92,157.67,152.44,153.88,153.15,156.80,155.65,153.99,...,148.28,142.50,146.09,157.93,156.18,156.05,156.30,146.59,2019-01-31,1
9635,165.66,160.54,155.92,157.67,152.44,153.88,153.15,156.80,155.65,153.99,...,148.28,142.50,146.09,157.93,156.18,156.05,156.30,146.59,2019-01-31,1


In [ ]:
aapl_dataset.drop(columns=['prediction_date'], inplace=True)

In [ ]:
Aapl_X, Aapl_Y = aapl_dataset.drop(columns=['direction']), (aapl_dataset['direction'] + 1) / 2

In [ ]:
Aapl_X

,prevClose_1,prevClose_2,prevClose_3,prevClose_4,prevClose_5,prevClose_6,prevClose_7,prevClose_8,prevClose_9,prevClose_10,...,prevClose_16,prevClose_17,prevClose_18,prevClose_19,prevClose_20,prevClose_21,prevClose_22,prevClose_23,prevClose_24,prevClose_25
5658,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,187.98,189.28,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08
5766,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,187.98,189.28,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08
5765,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,187.98,189.28,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08
5764,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,187.98,189.28,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08
5763,201.64,195.64,189.98,190.98,194.33,194.25,192.88,191.56,191.12,192.07,...,187.98,189.28,190.74,187.93,185.45,183.60,187.10,182.95,185.56,184.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9613,165.66,160.54,155.92,157.67,152.44,153.88,153.15,156.80,155.65,153.99,...,150.55,147.76,148.28,142.50,146.09,157.93,156.18,156.05,156.30,146.59
9614,165.66,160.54,155.92,157.67,152.44,153.88,153.15,156.80,155.65,153.99,...,150.55,147.76,148.28,142.50,146.09,157.93,156.18,156.05,156.30,146.59
9615,165.66,160.54,155.92,157.67,152.44,153.88,153.15,156.80,155.65,153.99,...,150.55,147.76,148.28,142.50,146.09,157.93,156.18,156.05,156.30,146.59
9635,165.66,160.54,155.92,157.67,152.44,153.88,153.15,156.80,155.65,153.99,...,150.55,147.76,148.28,142.50,146.09,157.93,156.18,156.05,156.30,146.59


In [ ]:
Aapl_Y

5658    1.0
5766    1.0
5765    1.0
5764    1.0
5763    1.0
       ... 
9613    1.0
9614    1.0
9615    1.0
9635    1.0
9583    1.0
Name: direction, Length: 10000, dtype: float64

In [ ]:
#from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(Aapl_X, Aapl_Y, test_size=0.2, random_state=56)

In [ ]:
TrainX = X_train.to_numpy().reshape((X_train.shape[0], X_train.shape[1], 1))

In [ ]:
TestX = X_test.to_numpy().reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
Y_train

5251    1.0
1401    1.0
422     1.0
3807    0.0
3389    0.0
       ... 
6369    1.0
6795    1.0
3075    0.0
5620    1.0
2807    0.0
Name: direction, Length: 8000, dtype: float64

In [ ]:
model = lstm_model(X_train.shape)

In [ ]:
model.fit(TrainX, Y_train, epochs=10, batch_size=64, verbose=1,validation_split=0.2)

Epoch 1/10
100/100 [==============================] - 7s 51ms/step - loss: 0.6794 - accuracy: 0.4397 - val_loss: 0.6814 - val_accuracy: 0.5400
Epoch 2/10
100/100 [==============================] - 5s 47ms/step - loss: 0.6674 - accuracy: 0.5145 - val_loss: 0.6896 - val_accuracy: 0.5750
Epoch 3/10
100/100 [==============================] - 5s 47ms/step - loss: 0.6602 - accuracy: 0.5533 - val_loss: 0.6960 - val_accuracy: 0.5750
Epoch 4/10
100/100 [==============================] - 5s 47ms/step - loss: 0.6537 - accuracy: 0.5747 - val_loss: 0.7061 - val_accuracy: 0.5750
Epoch 5/10
100/100 [==============================] - 5s 46ms/step - loss: 0.6531 - accuracy: 0.5723 - val_loss: 0.7149 - val_accuracy: 0.5750
Epoch 6/10
100/100 [==============================] - 5s 46ms/step - loss: 0.6502 - accuracy: 0.5730 - val_loss: 0.7193 - val_accuracy: 0.5750
Epoch 7/10
100/100 [==============================] - 5s 46ms/step - loss: 0.6515 - accuracy: 0.5734 - val_loss: 0.7045 - val_accuracy: 0.5750

In [ ]:
model.fit(TrainX, Y_train, epochs=20, batch_size=64, verbose=1,validation_split=0.2)

Epoch 1/20
100/100 [==============================] - 6s 55ms/step - loss: 0.6454 - accuracy: 0.5777 - val_loss: 0.7051 - val_accuracy: 0.5750
Epoch 2/20
100/100 [==============================] - 5s 45ms/step - loss: 0.6465 - accuracy: 0.5766 - val_loss: 0.7178 - val_accuracy: 0.5750
Epoch 3/20
100/100 [==============================] - 4s 45ms/step - loss: 0.6454 - accuracy: 0.5730 - val_loss: 0.7275 - val_accuracy: 0.5750
Epoch 4/20
100/100 [==============================] - 5s 45ms/step - loss: 0.6450 - accuracy: 0.5763 - val_loss: 0.6902 - val_accuracy: 0.5569
Epoch 5/20
100/100 [==============================] - 4s 45ms/step - loss: 0.6437 - accuracy: 0.5772 - val_loss: 0.7708 - val_accuracy: 0.5750
Epoch 6/20
100/100 [==============================] - 5s 46ms/step - loss: 0.6438 - accuracy: 0.5831 - val_loss: 0.7398 - val_accuracy: 0.5750
Epoch 7/20
100/100 [==============================] - 5s 46ms/step - loss: 0.6442 - accuracy: 0.5808 - val_loss: 0.7631 - val_accuracy: 0.5750

In [ ]:
eva = model.evaluate(TestX, Y_test, verbose=2)

63/63 - 1s - loss: 0.7775 - accuracy: 0.5680 - 1s/epoch - 21ms/step
